# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 3:20PM,246754,16,SSNA_Trans13_Week1,Sartorius Stedim North America,Released
1,Sep 19 2022 3:19PM,246753,16,SSNA_Trans12_Week1,Sartorius Stedim North America,Released
2,Sep 19 2022 3:17PM,246749,10,0085993218,ISDIN Corporation,Released
3,Sep 19 2022 3:17PM,246749,10,0085993232,ISDIN Corporation,Released
4,Sep 19 2022 3:17PM,246749,10,0085993270,ISDIN Corporation,Released
5,Sep 19 2022 3:17PM,246749,10,0085993276,ISDIN Corporation,Released
6,Sep 19 2022 3:17PM,246749,10,0085993331,ISDIN Corporation,Released
7,Sep 19 2022 3:17PM,246749,10,0085993332,ISDIN Corporation,Released
8,Sep 19 2022 3:17PM,246752,16,SSNA_Trans11_Week1,Sartorius Stedim North America,Released
9,Sep 19 2022 3:15PM,246750,16,SSNA_Trans10_Week1,Sartorius Stedim North America,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,246749,Released,6
25,246750,Released,1
26,246752,Released,1
27,246753,Released,1
28,246754,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246749,NaN,NaN,6.0
246750,NaN,NaN,1.0
246752,NaN,NaN,1.0
246753,NaN,NaN,1.0
246754,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246677,0.0,1.0,0.0
246701,0.0,0.0,2.0
246708,1.0,7.0,1.0
246712,0.0,4.0,60.0
246723,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246677,0,1,0
246701,0,0,2
246708,1,7,1
246712,0,4,60
246723,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246677,0,1,0
1,246701,0,0,2
2,246708,1,7,1
3,246712,0,4,60
4,246723,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246677,,1,
1,246701,,,2
2,246708,1,7,1
3,246712,,4,60
4,246723,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 3:20PM,246754,16,Sartorius Stedim North America
1,Sep 19 2022 3:19PM,246753,16,Sartorius Stedim North America
2,Sep 19 2022 3:17PM,246749,10,ISDIN Corporation
8,Sep 19 2022 3:17PM,246752,16,Sartorius Stedim North America
9,Sep 19 2022 3:15PM,246750,16,Sartorius Stedim North America
10,Sep 19 2022 3:15PM,246747,10,ISDIN Corporation
39,Sep 19 2022 3:05PM,246746,10,ISDIN Corporation
40,Sep 19 2022 3:04PM,246745,10,ISDIN Corporation
41,Sep 19 2022 3:02PM,246744,19,"GUSA Granules USA, Inc."
42,Sep 19 2022 2:51PM,246743,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 19 2022 3:20PM,246754,16,Sartorius Stedim North America,,,1
1,Sep 19 2022 3:19PM,246753,16,Sartorius Stedim North America,,,1
2,Sep 19 2022 3:17PM,246749,10,ISDIN Corporation,,,6
3,Sep 19 2022 3:17PM,246752,16,Sartorius Stedim North America,,,1
4,Sep 19 2022 3:15PM,246750,16,Sartorius Stedim North America,,,1
5,Sep 19 2022 3:15PM,246747,10,ISDIN Corporation,,,29
6,Sep 19 2022 3:05PM,246746,10,ISDIN Corporation,,,1
7,Sep 19 2022 3:04PM,246745,10,ISDIN Corporation,,,1
8,Sep 19 2022 3:02PM,246744,19,"GUSA Granules USA, Inc.",,,1
9,Sep 19 2022 2:51PM,246743,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 3:20PM,246754,16,Sartorius Stedim North America,1,,
1,Sep 19 2022 3:19PM,246753,16,Sartorius Stedim North America,1,,
2,Sep 19 2022 3:17PM,246749,10,ISDIN Corporation,6,,
3,Sep 19 2022 3:17PM,246752,16,Sartorius Stedim North America,1,,
4,Sep 19 2022 3:15PM,246750,16,Sartorius Stedim North America,1,,
5,Sep 19 2022 3:15PM,246747,10,ISDIN Corporation,29,,
6,Sep 19 2022 3:05PM,246746,10,ISDIN Corporation,1,,
7,Sep 19 2022 3:04PM,246745,10,ISDIN Corporation,1,,
8,Sep 19 2022 3:02PM,246744,19,"GUSA Granules USA, Inc.",1,,
9,Sep 19 2022 2:51PM,246743,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 3:20PM,246754,16,Sartorius Stedim North America,1,,
1,Sep 19 2022 3:19PM,246753,16,Sartorius Stedim North America,1,,
2,Sep 19 2022 3:17PM,246749,10,ISDIN Corporation,6,,
3,Sep 19 2022 3:17PM,246752,16,Sartorius Stedim North America,1,,
4,Sep 19 2022 3:15PM,246750,16,Sartorius Stedim North America,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 3:20PM,246754,16,Sartorius Stedim North America,1.0,NaN,NaN
1,Sep 19 2022 3:19PM,246753,16,Sartorius Stedim North America,1.0,NaN,NaN
2,Sep 19 2022 3:17PM,246749,10,ISDIN Corporation,6.0,NaN,NaN
3,Sep 19 2022 3:17PM,246752,16,Sartorius Stedim North America,1.0,NaN,NaN
4,Sep 19 2022 3:15PM,246750,16,Sartorius Stedim North America,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 3:20PM,246754,16,Sartorius Stedim North America,1.0,0.0,0.0
1,Sep 19 2022 3:19PM,246753,16,Sartorius Stedim North America,1.0,0.0,0.0
2,Sep 19 2022 3:17PM,246749,10,ISDIN Corporation,6.0,0.0,0.0
3,Sep 19 2022 3:17PM,246752,16,Sartorius Stedim North America,1.0,0.0,0.0
4,Sep 19 2022 3:15PM,246750,16,Sartorius Stedim North America,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2467337,59.0,15.0,1.0
15,246712,60.0,4.0,0.0
16,1727133,5.0,2.0,0.0
19,1233696,5.0,3.0,0.0
20,246742,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2467337,59.0,15.0,1.0
1,15,246712,60.0,4.0,0.0
2,16,1727133,5.0,2.0,0.0
3,19,1233696,5.0,3.0,0.0
4,20,246742,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,59.0,15.0,1.0
1,15,60.0,4.0,0.0
2,16,5.0,2.0,0.0
3,19,5.0,3.0,0.0
4,20,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,59.0
1,15,Released,60.0
2,16,Released,5.0
3,19,Released,5.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,15.0,4.0,2.0,3.0,0.0
Released,59.0,60.0,5.0,5.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,15.0,4.0,2.0,3.0,0.0
2,Released,59.0,60.0,5.0,5.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,15.0,4.0,2.0,3.0,0.0
2,Released,59.0,60.0,5.0,5.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()